In [8]:
import pandas as pd

In [25]:
class Datapoint():
    def __init__(self, km, price):
        self.km = km
        self.price = price

    def __str__(self):
        return (f"km: {self.km}, price {self.price}")


class Dataset():
    def __init__(self, data = None, path = None):
        self.data = data
        if (path != None):
            self.import_csv(path)

    def import_csv(self, path):
        self.data = pd.read_csv(path)

    def __getitem__(self, i):
        return Datapoint(self.data["km"][i], self.data["price"][i])
    
    def __len__(self):
        return (self.data.shape[0])

In [ ]:
class Irma():
    def __init__(self, dataset):
        self.dataset = dataset 
        self.theta0 = 0.0
        self.theta1 = 0.0
        self.learning_rate = 0.05
        self.minimal_improvement = 0.1

    def predict_price(self, km):
        return (self.theta0 + (theta1 * km))

    def error_datapoint(self, datapoint):
        error = self.predict_price(datapoint.km) - datapoint.price
        return (error)

    def squared_error_datapoint(self, datapoint):
        return (self.error_datapoint(datapoint)**2)
    
    def mean_squared_error_dataset(self):
        total_error = 0
        for i in range(len(self.dataset)):
            total_error += self.squared_error_datapoint(self.dataset[i])
        return (total_error / (2 * len(self.dataset)))
    
    def update_thetas(self):
        sum_errors_t0 = 0
        sum_errors_t1 = 0
        for i in len(self.dataset):
            sum_errors_t0 += self.error_datapoint(self.dataset[i])
            sum_errors_t1 += (self.error_datapoint(self.dataset[i]) * self.dataset[i].km)
        temp0 = self.theta0 - (self.learning_rate / len(self.dataset) * sum_errors_t0)
        temp1 = self.theta1 - (self.learning_rate / len(self.dataset) * sum_errors_t1)
        self.theta0 = temp0
        self.theta1 = temp1
    
    def compare_cost(self):
        oldcost = mean_squared_error_dataset()
        jesus_theta0 = self.theta0
        jesus_theta1 = self.theta1
        self.update_thetas()
        newcost = mean_squared_error_dataset()
        while (newcost <= oldcost):
            oldcost = newcost
            jesus_theta0 = self.theta0
            jesus_theta1 = self.theta1
            self.update_thetas()
            newcost = mean_squared_error_dataset()
        


In [29]:
datasetto =  Dataset(path = "data.csv")


240000
139800
150500
185530
176000
114800
166800
89000
144500
84000
82029
63060
74000
97500
67000
76025
48235
93000
60949
65674
54000
68500
22899
61789


In [14]:
df = pd.read_csv("data.csv")
df["km"][0]

240000

In [30]:
4**2

16